In [ ]:
!pip install git+https://github.com/openai/whisper.git
!apt-get install -y ffmpeg
!pip install pysrt


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0mpmr_p_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0mpmr_p_
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=e5f40579fcb9a9eae3f5555f09f3396651bf77d8ae23da686f515ae960272635
  Stored in directory: /root/.cache/pip/wheels/6a/36/54/2aa8dc961885d

In [ ]:
!pip install srt


  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=b2aab82f6fec48d9eab5bb03f1997804c9cdec1b4d2e2ec67f99d5b626f7b805
  Stored in directory: /root/.cache/pip/wheels/7e/75/5b/e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
Successfully built srt


In [ ]:
pip install faster-whisper


In [ ]:
from google.colab import files

uploaded = files.upload()  # upload your .mp4 file


Saving tester.mp4 to tester (1).mp4


In [ ]:
from faster_whisper import WhisperModel
import srt
from datetime import timedelta
import torch

# Load model
model = WhisperModel("small", device="cuda" if torch.cuda.is_available() else "cpu")

# Transcribe with word-level timestamps
segments, info = model.transcribe("tester.mp4", word_timestamps=True)

subs = []
idx = 1
for seg in segments:
    for word in seg.words:
        if word.word.strip():  # skip blanks
            subs.append(srt.Subtitle(
                index=idx,
                start=timedelta(seconds=word.start),
                end=timedelta(seconds=word.end),
                content=word.word.strip()
            ))
            idx += 1

# Save as SRT
srt_data = srt.compose(subs)
with open("word_level.srt", "w", encoding="utf-8") as f:
    f.write(srt_data)

print("✅ Word-level subtitles saved as word_level.srt")


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

✅ Word-level subtitles saved as word_level.srt


In [ ]:
def split_srt_with_timing(input_file, output_file, max_words=4):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    new_srt = []
    idx = 1
    i = 0

    while i < len(lines):
        if lines[i].isdigit():
            i += 1
            start_end = lines[i]
            start_str, end_str = start_end.split(" --> ")
            start_time = parse_timestamp(start_str)
            end_time = parse_timestamp(end_str)
            duration = (end_time - start_time).total_seconds()

            i += 1
            text_lines = []
            while i < len(lines) and lines[i] != "":
                text_lines.append(lines[i])
                i += 1

            full_text = " ".join(text_lines).strip()
            words = full_text.split()

            # Create chunks of words
            chunks = [words[j:j+max_words] for j in range(0, len(words), max_words)]
            chunk_duration = duration / len(chunks)

            for k, chunk_words in enumerate(chunks):
                chunk_start = start_time + timedelta(seconds=k * chunk_duration)
                chunk_end = chunk_start + timedelta(seconds=chunk_duration)
                chunk_ts = f"{format_timestamp(chunk_start)} --> {format_timestamp(chunk_end)}"
                chunk_text = " ".join(chunk_words)
                new_srt.append(f"{idx}\n{chunk_ts}\n{chunk_text}\n")
                idx += 1
        i += 1

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(new_srt))


In [ ]:
ffmpeg -i "tester.mp4" -vf "subtitles=word_level.srt:force_style='FontName=Arial,FontSize=28,PrimaryColour=&HFFFFFF&,BackColour=&H000000&,BorderStyle=3,Outline=1,Shadow=0,Alignment=2,MarginV=500'" -c:a copy "video_with_wordsubs.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from google.colab import files
files.download("video_with_wordsubs.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>